In [ ]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the folder contianing the subfolders of images.
# Function input arg 1: create_all_videos [bool] --> When 0, asks for the Well diirectory. When 1, asks for the directory containing the village folders.
# Function input arg 2: test [bool] --> When 1, will change the gui title to that of the test gui.
# Function output 1: The path of the folder selected by the user. 
def folder_selection_dialog(create_all_videos = 0,
                            test = 0):
    root = Tk()
    if test:
        root.title('Please select the "test" folder within the downloaded package.')
        root.filename = filedialog.askdirectory(initialdir="/", title="Please select the 'test' folder within the downloaded package.")
    else:
        if create_all_videos == 0:
            root.title('Please select the well directory for which you want to make a video')
            root.filename = filedialog.askdirectory(initialdir="/", title="Please select the directory containing the subfolders of image data")
        elif create_all_videos == 1:
            root.title('Please select the folder containing the village directories')
            root.filename = filedialog.askdirectory(initialdir="/", title="Please select the directory containing the subfolders of image data")
        else:
            raise Exception("Value of create_all_videos must be 0 or 1, and cannot be int type.")
    selected_directory = root.filename
    root.destroy()

    return selected_directory

selected_directory = folder_selection_dialog()

In [ ]:
import os as os

# A function to create a list of image paths, such that these images can later be used to make a movie. 
# Function input arg 1: selected_directory [string] --> The directory containing the subfolders of images. 
# Function input arg 2: file_type [string] --> The file extension that you want to detect. 
# Function output 1: txt_path [string] --> A string of the path of the txt containing  a list of the image paths.
def list_image_paths(selected_directory,
                     file_type = '.JPG'):
    
    # Create a new folder to store the sequentially named images. 
    if not os.path.exists(os.path.join(selected_directory, 'renamed_images')):
        os.makedirs(os.path.join(selected_directory, 'renamed_images'))
    
    # Create our txt file to store file names as they're coppied into the new folder. 
    txt_path = os.path.join(selected_directory, "image_paths.txt")

    with open(txt_path, 'w') as f:

        # First, list the subfolders in the well directory.
        subfolders = [_ for _ in os.listdir(selected_directory) if '.' not in _]

        # Iterate through the list of subfolders and extract the images contained within them. 
        for i in range(len(subfolders)):

            # Build the path to the DCIM folder for this iteration.
            current_subfolder = subfolders[i]
            subfolder_dir = os.path.join(selected_directory, current_subfolder, 'DCIM')

            # Detect the EK folders which contain the images. 
            EK_folders = [_ for _ in os.listdir(subfolder_dir) if '.' not in _]

            for u in range(len(EK_folders)):

                # Build the path to the current EK folder. 
                current_EK_folder = EK_folders[u]
                EK_folder_dir = os.path.join(subfolder_dir, current_EK_folder)

                # Collect the image paths from that EK folder. 
                images_in_EK_folder = [os.path.join(EK_folder_dir, image) for image in os.listdir(EK_folder_dir) if image.endswith(file_type)]

                #Iterate through each image, copy it to a single folder with ffmpeg-compatible names, then and add the new image path to the txt file. 
                image_number = 1 
                for p in range(len(images_in_EK_folder)):
                    
                    if not os.path.exists('my_folder'):
                        os.makedirs('my_folder')
                    f.write(f"file '{images_in_EK_folder[p]}'\n")
                    
                    image_number += 1
    
    # Close our txt file. 
    f.close()

    return txt_path

In [ ]:
import os as os

# A function to return the list of well folders, such that images with can later be used to make a movie. 
# Function input arg 1: selected_directory [string] --> The directory containing the well folders of images. 
# Function output 1: well_paths [list] --> A list of the well_folder paths, where each path is a string. 
def list_well_paths(selected_directory):
    
    # First, list the subfolders in the well directory.
    subfolders = [_ for _ in os.listdir(selected_directory) if '.' not in _]

    # Iterate through the list of village_directories and extract the well_directories contained within them. 
    well_paths = []
    for i in range(len(subfolders)):
        
        subfolder_dir = os.path.join(selected_directory, subfolders[i])
        well_directories = [os.path.join(subfolder_dir, _) for _ in os.listdir(subfolder_dir) if '.' not in _]
        well_paths.append(well_directories)
        
    well_paths = [path for sublist in well_paths for path in sublist]
    
    return well_paths

In [ ]:
import os 
import cv2
import time as time 
import math
import ffmpeg 
import subprocess

# A function to take the list of image paths, load in said images, and convert them to a movie. 
# Function input arg 1: selected_directory [string] --> The well or village directory, as previously selected. 
# Function input arg 1: create_all_videos [bool] --> When 0, creates individual videos from the well directory. When 1, considers every well directory and makes videos for all of them.
# Function input arg 3: frame_rate [int] --> The desired frame rate. 
# Function input arg 4: movie_extension [string] --> Your desired movie file extension. Tested for .avi and .mp4. 
# Function output 1: The movie will be saved to 'selected_directory'. 
def create_movie(selected_directory,
                 create_all_videos = 0,
                 file_type = '.JPG',
                 frame_rate = '6',
                 movie_extension = '.mp4'):
    
    if create_all_videos == 0:
    
        # Construct the movie name. 
        path_components = os.path.normpath(selected_directory).split(os.path.sep)
        n = len(path_components)
        movie_name = (f"{path_components[n-2]}_{path_components[n-1]}_fps{frame_rate}{movie_extension}")

        # Get the image paths. 
        image_paths = list_image_paths(selected_directory,
                                       file_type = file_type)
    
        # Change the current working directory to the well folder. 
        os.chdir(selected_directory)

        # Start recording the processing duration.
        t = time.time() 

        # Create the movie. 
        ffmpeg.input('image_paths.txt', pattern_type='glob', r=frame_rate, f='concat', safe='0').output(movie_name, vcodec='libx264', video_bitrate='10000k').run(capture_stdout=True, capture_stderr=True)
        
        # Remove the image_paths.txt file. 
        os.remove("image_paths.txt")
        
        # Calculate the processing duration.
        elapsed = time.time() - t 
        
        # Print the completion statement.
        print(f"------------\nYour video, '{movie_name}' (FPS = {frame_rate}), has been saved to:\n\n{selected_directory}\nTime taken = {elapsed/60} min\n------------\n\n")
        
    if  create_all_videos == 1: 
        
        # Get the well directories. 
        well_paths = list_well_paths(selected_directory)
        
        for v in range(len(well_paths)):
            
            # Construct the movie name. 
            path_components = os.path.normpath(well_paths[v]).split(os.path.sep)
            n = len(path_components)
            movie_name = (f"{path_components[n-2]}_{path_components[n-1]}_fps{frame_rate}{movie_extension}")
            
            # Get the image paths. 
            image_paths = list_image_paths(well_paths[v],
                                          file_type = file_type)

            # Change the current working directory to the well folder. 
            os.chdir(well_paths[v])

            # Create the movie.
            ffmpeg.input('image_paths.txt', r=frame_rate, f='concat', safe='0').output(movie_name, vcodec='libx264').run()

            # Remove the image_paths.txt file. 
            os.remove("image_paths.txt")

            # Calculate the processing duration.
            elapsed = time.time() - t 

            # Print the completion statement.
            print(f"------------\nYour video, '{movie_name}' (FPS = {frame_rate}), has been saved to:\n\n{selected_directory}\nTime taken = {elapsed/60} min\n------------\n\n")


In [ ]:
# Things to do. 
# 1 - Encrypt the drive.
# 2 - CHECK THE DATE ORDERS!!
# 3 - Sort out EK folder detection. 
# 4 - Look into video detection and stitching.
# 5 - Naming has broken.
# 6 - time recoding has broken. 
# 7 - Videos are huge. Make them smaller.


In [ ]:
# Testing ffmpeg

import os 
import ffmpeg 
import subprocess

os.chdir(selected_directory)

ffmpeg.input('image_paths.txt', r='20', f='concat', safe='0').output('test346.mp4', vcodec='libx264').run()
